In [4]:
## Import Libraries 
import numpy as np
import pandas as pd
from pandasql import sqldf
mysql = lambda q: sqldf(q, globals())
import sqlite3
from bs4 import BeautifulSoup
from selenium import webdriver
#from webdriver_manager.chrome import ChromeDriverManager
#driver = webdriver.Chrome(ChromeDriverManager().install())
import time

# Getting data from the web

In [ ]:
d = {}
for i in range(30):
    driver.get('https://www.udemy.com/topic/sql/'+'?p='+str(i+1))
    time.sleep(5)
    
    content = driver.page_source
    d["soup{0}".format(i)] = BeautifulSoup(content)
    print(i)

# Creating a list with the data extracted from the soup

In [42]:
courses = []

for i in d:
    for container in d[i].find('div', attrs={'class':'course-list--container--3zXPS'}):

        course = str(container.findAll('div', attrs={'class':
                                       'udlite-focus-visible-target udlite-heading-md course-card--course-title--2f7tE'}))[93:-7]


        author = str(container.findAll('div', attrs={'class':
                                       'udlite-text-xs course-card--instructor-list--lIA4f'}))[134:-7]


        rating = str(container.findAll('span', attrs={'class':
                                       'udlite-heading-sm star-rating--rating-number--3lVe8'}))[115:-8]


        review = str(container.findAll('span', attrs={'class':
                                       'udlite-text-xs course-card--reviews-text--12UpL'}))[1:-1]

        review = review[review.find("(")+1:review.find(")")]
        
        
        prices = container.findAll('div', attrs={'class':
                                      'price-text--price-part--Tu6MH course-card--discount-price--3TaBk udlite-heading-md'})
        
        for tag in prices:
            price = tag.findAll("span")[1]
            for i in price:
                price = tag.text[13:]
            
        
        #Hours, number of lessons and difficulty level
        if len(list(container.findAll("span",{'class' : 'course-card--row--1OMjg'}))) == 3:
            others = list(container.findAll("span",{'class' : 'course-card--row--1OMjg'}))
            
            hours = str(others[0])
            hours = hours[hours.find(">")+1:hours.find("</")]
            
            lessons = str(others[1])
            lessons = lessons[lessons.find(">")+1:lessons.find("</")]
            
            level = str(others[2])
            level = level[level.find(">")+1:level.find("</")]
            
        else: hours, lessons, level = np.nan,np.nan,np.nan


        courses.append([course,author,rating,review,hours,lessons,level,price])

# Creating a dataframe with pandas

In [37]:
column_names = ['course','author','rating','reviews','hours','lessons','level','price']
df = pd.DataFrame(courses, columns=column_names)

NameError: name 'courses' is not defined

In [124]:
df2 = df.dropna(axis=0).reset_index(drop='True')
df2

,course,author,rating,reviews,hours,lessons,level,price
0,SQL - Curso completo de Bases de Datos - de 0 ...,Pablo Tilotta - +110k students,"4,4",2.687,22 horas en total,106 clases,Todos los niveles,"14,99 €"
1,SQL. Curso completo de SQL. Aprende desde cero...,Redait Media,"4,4",2.301,"2,5 horas en total",77 clases,Todos los niveles,"14,99 €"
2,SQL desde cero: Curso práctico y sin prerrequi...,Mel Restori,"4,6",1.806,2 horas en total,17 clases,Todos los niveles,"16,99 €"
3,SQL Básico - Intermedio ¡Añade valor a tu CV e...,Numpi Cursos,"4,5",1.455,"1,5 horas en total",19 clases,Todos los niveles,"14,99 €"
4,Aprende SQL desde cero: ¡Curso con mas de 50 e...,Numpi Cursos,"4,3",1.372,"1,5 horas en total",67 clases,Todos los niveles,"14,99 €"
...,...,...,...,...,...,...,...,...
469,SQL: ничего лишнего,Валерий Жданов,"4,0",13,"1,5 horas en total",10 clases,Principiante,"14,99 €"
470,SQL - Data Analysis mit MySQL - Wochenendseminar,Karsten Brodmann,"4,6",12,"5,5 horas en total",42 clases,Todos los niveles,"14,99 €"
471,SQL Server : Création et modification d'une ba...,O.Thuillier | SQL | Postgres | Powershell| MyS...,"4,6",12,2 horas en total,29 clases,Intermedio,"14,99 €"
472,Sistema para Clinica Odontológica CSharp e SQL,Hugo Vasconcelos,"3,7",12,5 horas en total,57 clases,Todos los niveles,"14,99 €"


# Cleaning the data

In [ ]:
df2['hours'] = df2['hours'].str.replace(r'\D', '')
df2['lessons'] = df2['lessons'].str.replace(r'\D', '')
df2['price'] = df2['price'].str.replace(r'\xa0€', '')
df2['price'] = df2['price'].str.replace(',','.')
df2['reviews'] = df2['reviews'].str.replace(r'.', '')

In [130]:
df2['reviews'] = pd.to_numeric(df2['reviews']).fillna(0).astype(int)
df2['hours'] = pd.to_numeric(df2['hours']).fillna(0).astype(int)
df2['lessons'] = pd.to_numeric(df2['lessons']).fillna(0).astype(int)

# Saving the Dataframe to csv

In [39]:
#Saved the clean df so I don't need to do the webscrapping everytime I want to work on this
#df2.to_csv('sql_scrap_df.csv',index=False)

#Now we can read it from the file
sql_courses = pd.read_csv('sql_scrap_df.csv')
sql_courses['price'] = sql_courses['price'].str.replace(',','.')

# Analyzing the dataframe with pandas

In [40]:
# top 10 courses by reviews
sql_courses.sort_values(by=['reviews'],ascending=False).head(5)

,course,author,rating,reviews,hours,lessons,level,price
66,The Complete SQL Bootcamp 2021: Go from Zero t...,Jose Portilla,"4,7",101806,9,83,Todos los niveles,14.99
67,The Ultimate MySQL Bootcamp: Go from SQL Begin...,"Colt Steele, Ian Schoonover","4,6",55647,205,325,Principiante,14.99
330,Introduction to Databases and SQL Querying,Rakesh Gopalakrishnan,"4,5",36568,25,11,Principiante,Gratis
68,SQL - MySQL for Data Analytics and Business In...,365 Careers,"4,6",28842,10,287,Todos los niveles,14.99
356,"O curso completo de Banco de Dados e SQL, sem ...",Felipe Mafra,"4,6",26846,58,309,Principiante,14.99


In [42]:
# top 10 courses by hours
sql_courses.sort_values(by=['hours'],ascending=False).head(5)

,course,author,rating,reviews,hours,lessons,level,price
383,C#+SQL الدورة الشاملة لمطورى تطبيقات سطح المكت...,Mahmoud Bakr,"4,8",327,815,449,Todos los niveles,14.99
250,SQL Programming for Data Analysis and Software...,Ankit Srivastava,"3,9",13,255,119,Principiante,14.99
82,Complete SQL and Databases Bootcamp: Zero to M...,"Andrei Neagoie, Mo Binni","4,6",1627,245,278,Todos los niveles,14.99
368,Die komplette SQL Masterclass: Vom Anfänger zu...,"Jannis Seemann (CodingCourses.TV), Jannis Seemann","4,7",937,235,250,Principiante,14.99
241,Build Courier Tracking System in PHP OOP -- 2019,Peterson Fredrick,"3,8",17,225,85,Intermedio,14.99


In [43]:
# top 10 courses by price and reviews
sql_courses.sort_values(by=['price','reviews'],ascending=False).head(5)

,course,author,rating,reviews,hours,lessons,level,price
330,Introduction to Databases and SQL Querying,Rakesh Gopalakrishnan,"4,5",36568,25,11,Principiante,Gratis
331,Advanced Databases and SQL Querying,Rakesh Gopalakrishnan,"4,5",10840,35,27,Intermedio,Gratis
58,Aprende SQL desde CERO hasta AVANZADO como un ...,Candido Acevedo Medrano,"4,4",3376,95,91,Principiante,Gratis
59,SQL Server: Nivel 1 Sentencias Principales,Mariano Puglisi,"4,6",2602,15,34,Principiante,Gratis
332,Free SQL 101 Class Bootcamp Beginners Intro to...,Shivgan Joshi,"4,2",1496,15,17,Principiante,Gratis


In [44]:
# top 10 courses 'intermediate level' by reviews
sql_courses[sql_courses['level'] == 'Intermedio'].sort_values(by=['reviews'],ascending=False).head(5)

,course,author,rating,reviews,hours,lessons,level,price
331,Advanced Databases and SQL Querying,Rakesh Gopalakrishnan,"4,5",10840,35,27,Intermedio,Gratis
362,SQL - Bootcamp: Lerne MySQL in 2 Wochen,"Jannis Seemann, Jannis Seemann (CodingCourses.TV)","4,6",1461,75,108,Intermedio,14.99
89,SQL Tuning,Amarnath Reddy,"4,3",1123,3,61,Intermedio,14.99
367,Formação SQL Server 2017 Desenvolvedor Expert ...,André Rosa,"4,5",960,215,121,Intermedio,14.99
97,Sachin Quickly Learns (SQL) - Structured Query...,Kotaa Krishna Kamath,"4,4",620,15,13,Intermedio,14.99


In [45]:
sql_courses['price'].unique()

array(['14.99', '16.99', '17.99', '99.99', 'Gratis', '29.99', '199.99',
       '24.99', '19.99', '39.99'], dtype=object)

In [46]:
#Quantity of different course levels available
sql_courses.level.value_counts()

Principiante         253
Todos los niveles    162
Intermedio            53
Experto                6
Name: level, dtype: int64

# Analyzing the pandas dataframe using SQL 'pandasql'

In [65]:
#Prepare df for SQL
sql_courses = sql_courses.reset_index()
sql_courses = sql_courses.rename(columns={"index": "id"})

In [269]:
#Courses with more than 5000 reviews and longer than 25 hours
mysql('''SELECT * 
FROM sql_courses 
WHERE reviews > 5000 AND hours >= 25 
ORDER BY rating DESC;''')

,id,course,author,rating,reviews,hours,lessons,level,price
0,71,Master SQL For Data Science,Imtiaz Ahmad,"4,7",7779,95,51,Todos los niveles,"14,99"
1,67,The Ultimate MySQL Bootcamp: Go from SQL Begin...,"Colt Steele, Ian Schoonover","4,6",55647,205,325,Principiante,"14,99"
2,356,"O curso completo de Banco de Dados e SQL, sem ...",Felipe Mafra,"4,6",26846,58,309,Principiante,"14,99"
3,330,Introduction to Databases and SQL Querying,Rakesh Gopalakrishnan,"4,5",36568,25,11,Principiante,Gratis
4,331,Advanced Databases and SQL Querying,Rakesh Gopalakrishnan,"4,5",10840,35,27,Intermedio,Gratis


In [270]:
#Top rated free courses (at least 100 reviews)
mysql('''SELECT * 
FROM sql_courses 
WHERE rating > '4,5' AND price = 'Gratis' AND reviews > 100
ORDER BY rating DESC
LIMIT 10;''')

,id,course,author,rating,reviews,hours,lessons,level,price
0,64,Lo básico de SQL,Nacho Cabanes,"4,7",296,2,25,Principiante,Gratis
1,341,Step by Step Practical Oracle SQL with real li...,Ramesh R,"4,7",230,4,37,Principiante,Gratis
2,342,New in Big Data: Apache HiveMall - Machine Lea...,Elena Akhmatova,"4,7",213,32,7,Principiante,Gratis
3,59,SQL Server: Nivel 1 Sentencias Principales,Mariano Puglisi,"4,6",2602,15,34,Principiante,Gratis
4,61,Guía de iniciación a Bases de Datos con MS SQL...,Santi Santos Fuentes,"4,6",537,2,17,Principiante,Gratis
5,343,SQL Beginner's Level Course,Alexander Shafe,"4,6",133,2,5,Principiante,Gratis


In [59]:
#Top rated intermediate level courses, with at least 300 reviews and more than 25 hours to complete
mysql('''SELECT * 
FROM sql_courses 
WHERE level = 'Intermedio' AND reviews > 300 AND hours > 25
ORDER BY rating DESC
LIMIT 10;''')

,course,author,rating,reviews,hours,lessons,level,price
0,Alıştırmalarla SQL Öğreniyorum,"Ömer Çolakoğlu, Veri Bilimi Okulu","4,8",335,85,84,Intermedio,14.99
1,SQL - Beyond The Basics,Infinite Skills,"4,6",523,45,44,Intermedio,14.99
2,SQL - Bootcamp: Lerne MySQL in 2 Wochen,"Jannis Seemann, Jannis Seemann (CodingCourses.TV)","4,6",1461,75,108,Intermedio,14.99
3,Learn Advanced T-SQL For SQL Server Administrator,Raphael Asghar,"4,5",453,55,34,Intermedio,14.99
4,Advanced Databases and SQL Querying,Rakesh Gopalakrishnan,"4,5",10840,35,27,Intermedio,Gratis
5,Formação SQL Server 2017 Desenvolvedor Expert ...,André Rosa,"4,5",960,215,121,Intermedio,14.99
6,Project Based SQL Course: Code like a SQL Prog...,Brandon Brown,"4,0",450,65,41,Intermedio,14.99


# Creating SQL tables with sqlite3

In [5]:
conn = sqlite3.connect('test.db')

In [66]:
#push the dataframe to sql 

sql_courses.to_sql("sql_courses", conn, if_exists="replace")

In [ ]:
#Crear tabla con usuarios ficticios y otra con las compras de los cursos por esos usuarios

In [16]:
#Create table clientes
conn.execute('''DROP TABLE clientes;''') #Borra la tabla para poder volverla a crear si se ejecuta el codigo
conn.execute('''CREATE TABLE clientes(id,
    name, 
    lastname, 
    country, 
    age);''')
conn.commit()

In [22]:
#Insert values into a table clientes
conn.execute("DELETE FROM clientes;")#Borra los datos de la tabla para poder volverla a crear si se ejecuta el codigo
conn.execute('''INSERT INTO clientes 
      VALUES
      (0,'Gisela', 'Antolin', 'Spain', 30),
      (1,'Alex', 'Berrocal', 'Spain', 22),
      (2,'Claudia', 'Salguero', 'Argentina',27),
      (3,'Jenifer', 'Guevara', 'Colombia',31),
      (4,'Maialen', 'Poveda', 'Chile', 23),
      (5,'David', 'Ubeda', 'Spain', 54),
      (6,'Ascension', 'Dieguez', 'Spain', 22),
      (7,'Gisela', 'Calero', 'Ecuador', 33),
      (8,'Mohamed', 'Vidal', 'Spain', 40),
      (9,'Ignacio', 'Palacio', 'Spain', 27),
      (10,'Amador', 'Camino', 'Spain', 35);''')
conn.commit()

In [64]:
df_clientes = pd.read_sql_query("SELECT * FROM sql_courses", conn)
df_clientes

,index,course,author,rating,reviews,hours,lessons,level,price
0,0,SQL - Curso completo de Bases de Datos - de 0 ...,Pablo Tilotta - +110k students,"4,4",2687,22,106,Todos los niveles,14.99
1,1,SQL. Curso completo de SQL. Aprende desde cero...,Redait Media,"4,4",2301,25,77,Todos los niveles,14.99
2,2,SQL desde cero: Curso práctico y sin prerrequi...,Mel Restori,"4,6",1806,2,17,Todos los niveles,16.99
3,3,SQL Básico - Intermedio ¡Añade valor a tu CV e...,Numpi Cursos,"4,5",1455,15,19,Todos los niveles,14.99
4,4,Aprende SQL desde cero: ¡Curso con mas de 50 e...,Numpi Cursos,"4,3",1372,15,67,Todos los niveles,14.99
...,...,...,...,...,...,...,...,...,...
469,469,SQL: ничего лишнего,Валерий Жданов,"4,0",13,15,10,Principiante,14.99
470,470,SQL - Data Analysis mit MySQL - Wochenendseminar,Karsten Brodmann,"4,6",12,55,42,Todos los niveles,14.99
471,471,SQL Server : Création et modification d'une ba...,O.Thuillier | SQL | Postgres | Powershell| MyS...,"4,6",12,2,29,Intermedio,14.99
472,472,Sistema para Clinica Odontológica CSharp e SQL,Hugo Vasconcelos,"3,7",12,5,57,Todos los niveles,14.99


In [32]:
#Create table compras
conn.execute('''DROP TABLE compras;''')
conn.execute('''CREATE TABLE compras(id, 
    client_id, 
    course_id,
    payment, 
    Premium);''')
conn.commit()

In [85]:
#Insert values into a table compras
conn.execute("DELETE FROM compras;")
conn.execute('''INSERT INTO compras 
      VALUES
      (0, 1, 30, 49.99, 0),
      (1, 2, 83, 12.99, 0),
      (2, 6, 24, 9.99, 0),
      (3, 3, 4,  9.99, 0),
      (4, 5, 42, 14.99, 0),
      (5, 4, 89, 9.99, 0),
      (6, 2, 66, 14.99, 0),
      (7, 4, 71, 14.99, 0),
      (8, 7, 30, 19.99, 0),
      (9, 8, 92, 9.99, 0),
      (10, 8, 107, 14.99, 0),
      (11, 1, 52, 9.99, 0),
      (12, 6, 75, 12.99, 0),
      (13, 6, 174, 9.99, 0),
      (14, 3, 36,  9.99, 0),
      (15, 5, 73, 14.99, 0),
      (16, 3, 3, 9.99, 0),
      (17, 2, 148, 14.99, 0),
      (18, 4, 164, 14.99, 0),
      (19, 9, 78, 9.99, 0),
      (20, 8, 89, 9.99, 0),
      (21, 10, 90, 14.99, 0);''')
conn.commit()

In [58]:
df_compras = pd.read_sql_query("SELECT * FROM compras", conn)
df_compras

,id,client_id,course_id,payment,Premium
0,0,1,30,49.99,0
1,1,2,83,12.99,0
2,2,6,24,9.99,0
3,3,3,4,9.99,0
4,4,5,42,14.99,0
5,5,4,89,16.99,0
6,6,2,66,14.99,0
7,7,4,71,14.99,0
8,8,7,30,19.99,0
9,9,8,92,49.99,0


In [86]:
pd.read_sql_query('''SELECT cl.name,
                            cl.lastname,
                            co.payment,
                            sq.price,
                            (sq.price - co.payment) AS discounted
                            
                     FROM clientes cl
                     JOIN compras co ON co.client_id = cl.id
                     JOIN sql_courses sq ON sq.id = co.course_id;''', conn)

,name,lastname,payment,price,discounted
0,Alex,Berrocal,49.99,99.99,50.0
1,Claudia,Salguero,12.99,14.99,2.0
2,Ascension,Dieguez,9.99,14.99,5.0
3,Jenifer,Guevara,9.99,14.99,5.0
4,David,Ubeda,14.99,14.99,0.0
5,Maialen,Poveda,9.99,14.99,5.0
6,Claudia,Salguero,14.99,14.99,0.0
7,Maialen,Poveda,14.99,14.99,0.0
8,Gisela,Calero,19.99,99.99,80.0
9,Mohamed,Vidal,9.99,14.99,5.0
